In [594]:
import json
from telnetlib import EC
import requests
import shlex
import json
import subprocess
import urllib.request, urllib.error, urllib.parse
import urllib.request
import certifi
from io import BytesIO
from urllib.parse import urlencode
from typing import Generator, Any, Literal
from itertools import chain
import pandas as pd

def get_recebimento_nf(chave) -> Generator[Any, None, None]:

    valor = r"""curl -s https://app.omie.com.br/api/v1/produtos/recebimentonfe/ -H 'Content-type: application/json' -d '{"call":"ConsultarRecebimento","app_key":"1566467100198","app_secret":"8f7c2ebf7899831ecce7c488e69a3e33","param":[{"nIdReceb":0,"cChaveNfe":""}]}'"""
    new_val = valor.replace('cChaveNfe":""',f'cChaveNfe":"{chave}"')
    lCmd = shlex.split(new_val)
    p = subprocess.Popen(lCmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    json_data = json.loads(out.decode("utf-8"))
    yield json_data
   


def get_valores(valor) -> (Literal['list', 'DICT'] | None):
    match valor:
        case valor if type(valor)  == list:
            print("list")
            
        case valor if type(valor) == dict:
                dict_produtos: dict[str:str] = {}
                try:
                    if isinstance(valor, dict):
                        dict_produtos.update(valor)
                        yield dict_produtos
                except Exception as e:
                    print(e)
                    yield {"erro":e}
        case _:
            yield {"erro":e}
            

def notas_entrada_omie(pag: Any) -> Generator[Any, None, None]:
    valor = r"""curl -s https://app.omie.com.br/api/v1/produtos/recebimentonfe/ -H 'Content-type: application/json' -d '{"call":"ListarRecebimentos","app_key":"1566467100198","app_secret":"8f7c2ebf7899831ecce7c488e69a3e33","param":[{"nPagina":1,"nRegistrosPorPagina":2}]}'"""
    new_val = valor.replace('"nPagina":1,',f'"nPagina":{pag},')
    lCmd = shlex.split(new_val)
    p = subprocess.Popen(lCmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    out, err = p.communicate()
    json_data = json.loads(out.decode("utf-8"))
    yield json_data

listas_dict = []
for i in range(1, 10):
    jsons = notas_entrada_omie(i)

    for items in jsons:
        new_item = items.get('recebimentos')
        for new in new_item:
            item = new.keys()
           
            for key in item:
               
    
                nfentrada = new[key]
                
                if isinstance(nfentrada.get('cChaveNFe'), str):
                    chave_nfe = nfentrada.get('cChaveNFe')
                    jsons = next(get_recebimento_nf(chave_nfe))
                    dict_nf = {}
                    
            
                    cabec = jsons['cabec']
                   
                    itensRecebimento = jsons['itensRecebimento']
                    for item in itensRecebimento:
                        
                        dict_nf = item
                        dict_nf['cChaveNFe'] = cabec['cChaveNFe']
                        dict_nf['cNumeroNFe'] = cabec['cNumeroNFe']
                        dict_nf['cRazaoSocial'] = cabec['cRazaoSocial']
                        dict_nf['dEmissaoNFe'] = cabec['dEmissaoNFe']
                        dict_nf['nValorNFe'] = cabec['nValorNFe']
                    listas_dict.append(dict_nf)
                       
data = pd.DataFrame(listas_dict)
data.to_csv("impostos.csv")


In [ ]:
{'cCodItemInt': '', 'cCodProdInt': '', 'nCodItem': 5900966604, 'nCodProd': 2257454924,
 'CFOP': '6.108', 'EXTIPI': '', 'NCM': '6907.21.00', 'cCOFINSRetido': 'N', 'cCSLLRetido': 'N', 
 'cEAN': '7894287915361', 'cEANTrib': '', 'cINSSRetido': 'N', 'cIRRetido': 'N', 'cISSRetido': 'N', 
 'cOrigem': '0', 'cPISRetido': 'N', 'cProd': '25014A', 'cProdOrig': '', 
 'codigo_local_estoque': 2270778571, 'indTot': '1', 'nCMCTotal': 16511.002442,
 'nCMCUnitario': 38.87686, 'pCOFINS': 7.6, 'pICMS': 7, 'pMVAST': 0, 'pPIS': 1.65,
 'pRedBC': 0, 'qCom': 1.52, 'qTrib': 0, 'uCom': 'M2', 'uTrib': '', 'vBC': 249.41, 
 'vBCCOFINS': 139.41, 'vBCIPI': 0, 'vBCISS': 0, 'vBCPIS': 139.41, 'vBCST': 0,
 
 'vBCUFDest': 249.41, 'vCOFINS': 10.6, 'vDesc': 40.54, 'vFrete': 110, 'vICMS': 17.46,
 'vOutro': 0, 'vPIS': 2.3, 'vProd': 139.41, 'vSeg': 0, 'vTotItem': 139.41, 'vUnCom': 91.72,
 'vUnTrib': 0, 'xProd': 'Porcelanato Wood Decor HD 50x100,7cm Itagres', 'xProdOrig': '',
 'CodigoPedido': 4623, 'NumPedidoOmie': 841, 'NomeCliente': 'Adailson Felix da Silva', 
 'Uf': 'MG', 'Nome': 'Minas Gerais', 'NumeroNF': 584, 'Endereco': 'Avenida Adalardo Rios',
 'Numero': '230', 'Bairro': 'Loteamento Residencial Condomínio Villágio II', 'Cep': '38184550',
 'CustoUnitario': None,
 'IdProduto': 19024, 'SKU': '1040001002937', 'Quantidade': Decimal('23.3200')
 , 'QtdCaixa': 11}